### This will be in the prepare module
    _Check for and remove duplicates by customer_id. - None
    
    _Remove redundant columns: payment_type_id, internet_service_type_id, contract_type_id, & customer_id. - Completed
        -Used the following to drop redundant columns
        -df = df.drop(['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], axis = 1)
        -df.head(2)
    
    _Encode payment_type(Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)), internet_service_type(DSL, Fiber, None), contract_type(Month-to-month, One year, Two year), gender(male/female), partner(yes/no), multiple_line(yes/no), device_protection(yes/no), tech_support(yes/no), streaming_tv(yes/no), streaming_movies(yes/no), paperless_billing, & churn (yes/no).
    - gender is male(1) or not male(0)
    - yes(1) no(0)
    - payment, contract and service types (True = 1) (Fales = 0)
    
    _Check for nulls and fill (Fill in total_charges' blanks with 0). - Completed
        -#find empty values. Tried isna, isnull, notna, notnull but did not shown any matches.
            -df.eq(' ').sum()
        -Used .replace to empty values with a 0 in total charges.
            -df = df.replace({'total_charges': ' '}, 0)
            -df.head()

    _Convert total_charges to float64. - Completed
        -#convert 'total_charges' to float and validate change.
        - df['total_charges'] = df['total_charges'].astype(float)
        - df.dtypes
    
    _Replace 'No phone service' & 'No internet service' with 'No'. - Completed
        -df.replace(to_replace = 'No internet service', value = 'No')
        -df.replace(to_replace = 'No phone service', value = 'No')

In [1]:
#libraries
import acquire
import explore
import prepare
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

import graphviz
from graphviz import Graph

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Working code for project

# Acquire

In [2]:
df = acquire.get_telco_data()
df.head(1)

,payment_type_id,payment_type,internet_service_type_id,internet_service_type,contract_type_id,contract_type,customer_id,gender,senior_citizen,partner,...,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn
0,1,Electronic check,1,DSL,1,Month-to-month,0015-UOCOJ,Female,1,No,...,Yes,No,No,No,No,No,Yes,48.2,340.35,No


In [ ]:
#data summarization
df.describe(include='all').T

In [ ]:
df.info()

#### Initial visualization

In [ ]:
#plot charts on acquired data prior to data cleaning.
num_cols = df.columns[[df[col].dtype == 'int64' for col in df.columns]]
for col in num_cols:
    plt.hist(df[col])
    plt.title(col)
    plt.show()

In [ ]:
#value counts on acquired data prior to data cleaning.
obj_cols = df.columns[[df[col].dtype == 'O' for col in df.columns]]
for col in obj_cols:
    print(df[col].value_counts())
    print(df[col].value_counts(normalize=True, dropna=False))
    print('----------------------')

#### From above:
    - From above plot charts and column value counts we find the following have no weight on churn rate and will be dropped:
        - Remove 'phone_service', 'internet_service', 'senior_citizen', 'paperless' & 'tech_support' (irrelevant)

# Clean

In [3]:
#added above to the prepare module.
df = prepare.clean_data(df)
df.head(1)

,tenure,monthly_charges,dsl,fiber,no_internet,m2m,churned,partner,dependents,phone_service,tech_support,streaming_tv,streaming_movies,autopayment,not_autopayment
0,7,48.2,1,0,0,1,0,0,0,1,0,0,0,0,1


In [ ]:
df.info()

# Split your data into train, validate, and test samples.

In [4]:
train, validate, test = prepare.train_validate_test_split(df, 'churned')

In [5]:
print(train.shape, validate.shape, test.shape)

(3943, 15) (1691, 15) (1409, 15)


In [8]:
# create X & y version of train, where y is a series with just the target variable and X are all the features.
X_train = train.drop(columns=["churned"])
y_train = train.churned

X_validate = validate.drop(columns=["churned"])
y_validate = validate.churned

X_test = test.drop(columns=["churned"])
y_test = test.churned

In [12]:
# The mode is a great baseline
baseline = y_train.mode()

# Produce a boolean array with True representing a match between the baseline prediction and reality
matches_baseline_prediction = y_train == 0

baseline_accuracy = matches_baseline_prediction.mean()
print(f"Baseline accuracy: {round(baseline_accuracy, 2)}")

Baseline accuracy: 0.73


# Exploratory

#### Univariate Stats

In [ ]:
cat_vars = ['churned','tenure', 'dsl', 'fiber', 'no_internet', 'm2m', 'partner', 'dependents',
            'phone_service', 'tech_support', 'streaming_tv', 'streaming_movies', 'autopayment', 'not_autopayment']
quant_vars = ['monthly_charges']

In [ ]:
explore.explore_univariate(train, cat_vars, quant_vars)

    _Churn rate
    
    _Dropped 'gender' almost an equal amout of male and female which will not effect churn rate
    
    _Dropped 'online_security', 'device_protection' - small amount of customers use these services.
    
    _Seems like a large amount of people use 'tech_support' - Is churn rate affected by bad service.
    _Large number of customers use streaming services.
    
    

#### Bivariate Stats

In [ ]:
cat_vars = cat_vars[1:]

In [ ]:
explore.explore_bivariate(train, 'churned', cat_vars, quant_vars)

    _DROPPED one and two year contract. Numbers have no variance on churn.
    _DROPPED 'multiple_lines' has the similar weight which will not affect churn.
    _Validates drops 'online_security', 'device_protection'
    _DROPPED 'total_charges'. total charges for churn are lower than those who did not churn.
    
    _Single (687) customers churn more then couples
    _No dependents (863) churn more than people with dependents.
    -Do singles have an affect on churn? If so, why?
    
    _Lose customers in the first year.
    _Number of fiber (728)  customers who churned - what is about our fiber internet service is making customers churn?
    _Can we convert no_internet (779) to internet users?
    _Higher number of m2m (929) customers churn.
    _Those with phone_service (951) churn.
    _Customers who do not use 'tech_support' (862) tend to churn.
    _Customers who are not on auto_payment (780) churn.
    _Higher monthly cost have a higher tendency to churn and pay an averge of 74.3 than those who do not 61.85.. Does  higher monthly cost affect churn?

#### Multivariate Stats

    _ Need to drop some columns for the multivariate stats.

In [ ]:
target = 'churned'
explore.explore_multivariate(train, target, cat_vars, quant_vars)

# Modeling

### Decision Tree

In [13]:
#create the model

#for classification you can change the algorithm to gini or entropy (information gain).  
#Default is gini.

model = DecisionTreeClassifier(max_depth=1, random_state=123)

In [14]:
#Fit the model

model = model.fit(X_train, y_train)

In [15]:
#Use the model

# We'll evaluate the model's performance on train, first

y_predictions = model.predict(X_train)

In [16]:
# Produce the classification report on the actual y values and this model's predicted y values
report = classification_report(y_train, y_predictions, output_dict=True)
print("Tree of 1 depth")
pd.DataFrame(report)

Tree of 1 depth


,0,1,accuracy,macro avg,weighted avg
precision,0.734720,0.0,0.73472,0.367360,0.539813
recall,1.000000,0.0,0.73472,0.500000,0.734720
f1-score,0.847076,0.0,0.73472,0.423538,0.622363
support,2897.000000,1046.0,0.73472,3943.000000,3943.000000


### Random Forrest

In [17]:
#Create the Model
rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', min_samples_leaf=1, n_estimators=100,
                            max_depth=10, 
                            random_state=123)

In [18]:
#Fit the Model
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=123)

In [19]:
#Use the Model
#evaluate the weight
print(rf.feature_importances_)

[0.31363403 0.24757534 0.02067858 0.0625207  0.02509912 0.1503111
 0.02640062 0.02549778 0.01179398 0.03107333 0.01985905 0.01918382
 0.02153619 0.02483638]


In [20]:
#estimate
y_pred = rf.predict(X_train)

In [21]:
#probability
y_pred_proba = rf.predict_proba(X_train)

In [22]:
#accuracy
print('Accuracy of random forest classifier on training set: {:.2f}'.format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.89


In [23]:
#confusion matrix
print(confusion_matrix(y_train, y_pred))

[[2738  159]
 [ 289  757]]


In [24]:
#Create a classificaiton report
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92      2897
           1       0.83      0.72      0.77      1046

    accuracy                           0.89      3943
   macro avg       0.87      0.83      0.85      3943
weighted avg       0.88      0.89      0.88      3943



In [25]:
print('Accuracy of random forest classifier on test set: {:.2f}'.format(rf.score(X_validate, y_validate)))

Accuracy of random forest classifier on test set: 0.78


### KNN

In [26]:
#Create the model
# weights = ['uniform', 'density']
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [27]:
#fit the model
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [28]:
#predict/use the model
y_pred = knn.predict(X_train)

In [29]:
#probability
y_pred_proba = knn.predict_proba(X_train)

In [30]:
#accuracy
print('Accuracy of KNN classifier on training set: {:.2f}'.format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.84


In [31]:
#confusion matrix
print(confusion_matrix(y_train, y_pred))

[[2675  222]
 [ 414  632]]


In [32]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89      2897
           1       0.74      0.60      0.67      1046

    accuracy                           0.84      3943
   macro avg       0.80      0.76      0.78      3943
weighted avg       0.83      0.84      0.83      3943



In [33]:
#validate
print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_validate, y_validate)))

Accuracy of KNN classifier on test set: 0.76


# Notes and test code.

In [ ]:
#used to test adding two columns and renaming it.
df['auto_payment'] = df['bank_transfer'] + df['credit_card']

In [ ]:
#split data
def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [ ]:
df = prepare.train_validate_test_split(df, 'churned')

In [ ]:
#split into train, validate, test
train, validate, test = train_validate_test_split()
train.head(2)

In [ ]:
print(train.shape, validate.shape, test.shape)

In [ ]:
df.info()

In [ ]:
#Looking for unique values in "types"
df.contract_type.unique()

In [ ]:
#check to see if there are any duplicates by 'customer_id'
df.duplicated(subset = 'internet_service').unique()

In [ ]:
#find empty values. Tried isna, isnull, notna, notnull but did not shown any matches.
df.eq(' ').sum()

In [ ]:
#replaces empty values with a 0 in total charges.
df = df.replace({'total_charges': ' '}, 0)
df.head()

In [ ]:
#verify empty values in total_charges have been replace.
df.eq(' ').sum()

In [ ]:
df = df.drop(['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], axis = 1)
df.head(2)

In [ ]:
df.shape

In [ ]:
#convert 'total_charges' to float and validate change.
df['total_charges'] = df['total_charges'].astype(float)
df.dtypes

In [ ]:
df = df.replace(to_replace = 'No internet service', value = 'No')
df

In [ ]:
df = df.replace(to_replace = 'No phone service', value = 'No')
df

# Encode

In [ ]:
#get_dummies creates a seperate df of booleans for the identified columns below. Cleaning for the decission tree.
dummy_df = pd.get_dummies(df[['dependents','phone_service','online_security','online_backup','payment_type','internet_service_type','contract_type','gender','partner','multiple_lines','device_protection','tech_support','streaming_tv','streaming_movies','paperless_billing','churn']], dummy_na=False, drop_first=[True, True])
#set 'drop_first' to 'False' to encode multiple types of the below listed columns.
dummy_df_types = pd.get_dummies(df[['payment_type','internet_service_type','contract_type',]], dummy_na=False, drop_first=False)

In [ ]:
#now drop the above two columns...
df = df.drop(columns=['dependents','phone_service','online_security','online_backup','payment_type','internet_service_type','contract_type','gender','partner','multiple_lines','device_protection','tech_support','streaming_tv','streaming_movies','paperless_billing','churn'])
#...and concatanate the dummies df with the prep's df.
df = pd.concat([df, dummy_df, dummy_df_types], axis=1)
df.head()